This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [10]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [8]:
# First, import the relevant modules
import requests
import numpy as np

In [39]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/WIKI/FB.json?column_index=4&start_date=2014-01-01&end_date=2014-01-01&collapse=daily&transform=rdiff&api_key=' + API_KEY )

In [ ]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [143]:
#1.Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key=' + API_KEY )


In [144]:
#2.Convert the returned JSON object into a Python dictionary.
fse_dict = r.json()['dataset_data']

In [145]:
#3.Calculate what the highest and lowest opening prices were for the stock in this period.

#print(fse_dict['column_names'])
fse_list = fse_dict['data']

# clean list with replacine none value with 0
fse_list_clean = [[np.nan if x is None else x for x in c] for c in fse_list]
 
highest = max(fse_list_clean, key=lambda x: x[1])
lowest =  min(fse_list_clean, key=lambda x: x[1])

print('The highest opening value was: ' + str(highest[1]) + ' on ' + str(highest[0])) 
print('The lowest opening value was: ' + str(lowest[1]) + ' on ' + str(lowest[0]))




The highest opening value was: 53.11 on 2017-12-14
The lowest opening value was: 34.0 on 2017-01-24


In [146]:
#4.What was the largest change in any one day (based on High and Low price)?

change = max(fse_list_clean, key=lambda x: x[2]-x[3])
print('The largest change in value was: ' + str(round(change[2]-change[3],2)) + ' on ' + str(change[0]))


The largest change in value was: 2.81 on 2017-05-11


In [193]:
#5.What was the largest change between any two days (based on Closing Price)?

#all closing value from row 0 to end
fse_list_closing = [item[4] for item in fse_list_clean]

#all closing value excluding the first on ,  from row 1 to end
fse_list_closing_1 = [item[4] for item in fse_list_clean[1:]]


max_diff = max([abs(x1 - x2) for (x1, x2) in zip(fse_list_closing, fse_list_closing_1)])

print("The largest change between any two days (based on Closing Price) was ${0:.2f}." \
      .format(round(max_diff, 2)))

The largest change between any two days (based on Closing Price) was $2.56.


In [209]:
#6.What was the average daily trading volume during this year?
#print(fse_dict['column_names'])

from functools import reduce

fse_list_traded_volume = [item[6] for item in fse_list_clean]
traded_volume_mean = reduce(lambda x, y: x + y, fse_list_traded_volume) / len(fse_list_traded_volume)

print('The mean trading volume in 2017 was: ' + str(round(traded_volume_mean, 2)))

The mean trading volume in 2017 was: 89124.34


In [214]:
#7.(Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

#print(fse_dict['column_names'])

def calculate_median(ls):
    ls = sorted(ls)
    ls_len = len(ls)
    if ls_len < 1:
        return None
    if ls_len % 2 == 0 :
        return ( ls[(ls_len-1)//2] + ls[(ls_len+1)//2] ) / 2
    else:
        return ls[(ls_len-1)//2]
    
print('The median trading volume in 2017 was: ' + str(round(calculate_median(fse_list_traded_volume), 2)))

The median trading volume in 2017 was: 76286.0
